In [21]:
from chromedriver_py import binary_path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException

from joblib import Parallel, delayed

import tqdm

import numpy as np
import pandas as pd

from datetime import datetime

from PIL import Image
from urllib.request import urlopen,Request
from io import BytesIO

import time
import os

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [19]:
ruta = os.getcwd()+'/output/'

In [20]:
ruta

'/Users/cevalenciam/Library/CloudStorage/OneDrive-Personal/SmartBuy/smartbuy/web-scraper-lider/output/'

In [22]:
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    s = ChromeService(binary_path)
    driver = webdriver.Chrome(service=s, options=chrome_options)
    driver.maximize_window()
    return driver

In [23]:
def get_url_lider():
    
    url_lider = 'https://www.lider.cl/supermercado'

    return(url_lider)

In [24]:
def get_info_category():

    driver = get_driver()
    
    #Vamos a la dirección web de la página objetivo
    driver.get(get_url_lider())

    #Click en categorías
    try:
        boton = wait(driver, 4).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[@id='root']/div[@class='wrapper']/div/header/div[@class='header__center-content--desktop']/div[@class='header__menu--desktop']/button")))
        boton.click()
    except TimeoutException:
        print("Botón categorías no encontrado")

    list_categories = []

    #Espera de 5 seg
    time.sleep(5)

    #Obtener las categorías
    try:
        wait(driver, 8).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/div/div/div")))
        first_level_categories = driver.find_elements("xpath", "/html/body/div[5]/div/div/div/div/div")
        
        for first in first_level_categories:
            
            first_level_category = first.text

            #scroll para visualizar
            driver.execute_script("arguments[0].scrollIntoView();", first)
            
            #click en la categoria
            first.click()

            wait(driver, 8).until(ec.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div/div[2]/div[2]/div")))
            second_level_categories = driver.find_elements("xpath", "/html/body/div[5]/div/div/div/div[2]/div[2]/div")

            for second in second_level_categories:
                
                wait(second, 4).until(ec.presence_of_element_located((By.XPATH, "a")))
                second_level_category = second.find_elements("xpath", "a")[0].text
                url_first_level_category = second.find_elements("xpath", "a")[0].get_attribute("href")

                third_level_categories = second.find_elements("xpath", "div")

                for third in third_level_categories:
                    
                    wait(third, 4).until(ec.presence_of_element_located((By.XPATH, "a")))
                    third_level_category = third.find_elements("xpath", "a")[0].text
                    url_third_level_category = third.find_elements("xpath", "a")[0].get_attribute("href")

                    list_categories.append([first_level_category, second_level_category, url_first_level_category, third_level_category, url_third_level_category])
            
            
    except TimeoutException:
        print("Categorias no encontradas")
    
    driver.close()
    
    return(pd.DataFrame(list_categories, columns = ['first_level_category', 'second_level_category', 'url_second_level_category', 'third_level_category', 'url_third_level_category']), len(list_categories))

In [25]:
df_url_cat, num_cat = get_info_category()

In [26]:
categories = df_url_cat['first_level_category'].unique()

In [27]:
categories[15:]

array(['Mascotas', 'Librería y Cumpleaños', 'Tecno y Electro',
       'Ferretería', 'Automóvil', 'Vestuario', 'Parrillas y Jardín',
       'Deporte y Aire Libre', 'Marcas Propias', 'Llevar a', 'Campañas'],
      dtype=object)

In [40]:
df_url_cat

,first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category
0,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Cortas,https://www.lider.cl/supermercado/category/Des...
1,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Largas,https://www.lider.cl/supermercado/category/Des...
2,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Lasañas y Rellenos,https://www.lider.cl/supermercado/category/Des...
3,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Instantáneas,https://www.lider.cl/supermercado/category/Des...
4,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Integrales,https://www.lider.cl/supermercado/category/Des...
...,...,...,...,...,...
801,Campañas,Precios Que No Se Tocan,https://www.lider.cl/supermercado/category/Cam...,Carnes y Cecinas,https://www.lider.cl/supermercado/category/Cam...
802,Campañas,Precios Que No Se Tocan,https://www.lider.cl/supermercado/category/Cam...,Perfumería y Bebé,https://www.lider.cl/supermercado/category/Cam...
803,Campañas,Precios Que No Se Tocan,https://www.lider.cl/supermercado/category/Cam...,Frescos y Lácteos,https://www.lider.cl/supermercado/category/Cam...
804,Campañas,Precios Que No Se Tocan,https://www.lider.cl/supermercado/category/Cam...,Limpieza del Hogar,https://www.lider.cl/supermercado/category/Cam...


In [31]:
num_cat

806

In [11]:
df_url_cat.head(3)

,first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category
0,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Cortas,https://www.lider.cl/supermercado/category/Des...
1,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Largas,https://www.lider.cl/supermercado/category/Des...
2,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Lasañas y Rellenos,https://www.lider.cl/supermercado/category/Des...


In [20]:
actual_date = str(datetime.now())[0:10]

df_url_cat.to_excel(ruta + f'/df_categorias_lider.xlsx'.format(actual_date), index=0)

In [34]:
def get_pages_category(pais:str, i:str, first:str, second:str, third:str):

    driver = get_driver()

    total_url_cat = list()

    driver.get(i)

    try:

        wait(driver, 32).until(ec.presence_of_element_located((By.XPATH, "/html/body/div[@id='root']/div[@class='wrapper']/div[1]/div[2]/div[@class='app-container app-container--category-page']/div[@class='bs__grid__container']/div[@class='shop-wrapper']/div[@class='app-container--category-page__walstore-back-button-container']/div[@class='mb-4 ']/div[@class='shop-wrapper']/div[@class='d-flex']/div[@class='col-lg-9 col-md-8 bs__product-list__container']/div[@class='products-qantity-and-order-desktop shop-content']/div[@class='products-qantity-and-order-desktop__quantity-shown']")))
        per_page_count = driver.find_elements("xpath", "/html/body/div[@id='root']/div[@class='wrapper']/div[1]/div[2]/div[@class='app-container app-container--category-page']/div[@class='bs__grid__container']/div[@class='shop-wrapper']/div[@class='app-container--category-page__walstore-back-button-container']/div[@class='mb-4 ']/div[@class='shop-wrapper']/div[@class='d-flex']/div[@class='col-lg-9 col-md-8 bs__product-list__container']/div[@class='products-qantity-and-order-desktop shop-content']/div[@class='products-qantity-and-order-desktop__quantity-shown']")[0].text
        per_page_count_list = [int(s) for s in re.findall(r'-?\d+\.?\d*', per_page_count)]
        per_page = per_page_count_list[1]
        count = per_page_count_list[2]
        n_paginas = int(np.ceil(count/per_page))
        
        if n_paginas>1:
            url_cat_page = [[i + '?page=' + str(j) + '&hitsPerPage=16', first, second, third] for j in range(2, n_paginas+1)]
            total_url_cat.append([i, first, second, third])
            total_url_cat.extend(url_cat_page)
        else:
            total_url_cat.append([i, first, second, third])
    
    except:
        total_url_cat.append([i, first, second, third])
        
    driver.close()
    
    return(total_url_cat)

In [22]:
(df_url_cat.url_third_level_category[0], df_url_cat.first_level_category[0], df_url_cat.second_level_category[0], df_url_cat.third_level_category[0])

('https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas/Pastas_Cortas',
 'Despensa',
 'Pastas y Salsas',
 'Pastas Cortas')

In [23]:
get_pages_category('CL', df_url_cat.url_third_level_category[0], df_url_cat.first_level_category[0], df_url_cat.second_level_category[0], df_url_cat.third_level_category[0])

[['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas/Pastas_Cortas',
  'Despensa',
  'Pastas y Salsas',
  'Pastas Cortas'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas/Pastas_Cortas?page=2&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'Pastas Cortas'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas/Pastas_Cortas?page=3&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'Pastas Cortas'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas/Pastas_Cortas?page=4&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'Pastas Cortas'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas/Pastas_Cortas?page=5&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'Pastas Cortas'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas/Pastas_Cortas?page=6&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'Pastas Cortas']]

In [15]:
(df_url_cat['first_level_category']+df_url_cat['second_level_category']).nunique()

131

In [32]:
def get_pages_categories(pais:str, df_categories:pd.DataFrame):

    list_res = list()

    results = Parallel(n_jobs=-1)(delayed(get_pages_category)(pais, i, first, second, third) for i, first, second, third in tqdm.tqdm(zip(df_categories.url_third_level_category, df_categories.first_level_category, df_categories.second_level_category, df_categories.third_level_category)))
    
    for res in results:
        list_res = list_res + res
    
    return(pd.DataFrame(list_res, columns=['url_category', 'first_level_category', 'second_level_category', 'third_level_category']).drop_duplicates().values.tolist())

In [31]:
df_url_cat[(df_url_cat['third_level_category']=='ver todo') & (df_url_cat['first_level_category']=='Despensa')]

,first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category
7,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...
11,Despensa,Harinas y Polvos,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...
17,Despensa,Arroz y Legumbres,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...
23,Despensa,Salsas,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...
29,Despensa,Aceites y Aderezos,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...
38,Despensa,Cóctel y Snack,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...
43,Despensa,Conservas,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...
49,Despensa,Alimentos Instantáneos,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...
57,Despensa,Comida Étnica,https://www.lider.cl/supermercado/category/Des...,ver todo,https://www.lider.cl/supermercado/category/Des...


In [35]:
list_pages_categories = get_pages_categories('CL', df_url_cat[df_url_cat['third_level_category']=='ver todo'].iloc[:5, :])

5it [00:00, 4105.62it/s]


In [18]:
list_pages_categories

[['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas?page=2&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas?page=3&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas?page=4&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas?page=5&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas?page=6&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/category/Despensa/Pastas_y_Salsas?page=7&hitsPerPage=16',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['http

In [21]:
pd.DataFrame(list_pages_categories, columns=['url', 'first_category', 'second_category', 'third_category'])

,url,first_category,second_category,third_category
0,https://www.lider.cl/supermercado/category/Des...,Despensa,Pastas y Salsas,ver todo
1,https://www.lider.cl/supermercado/category/Des...,Despensa,Pastas y Salsas,ver todo
2,https://www.lider.cl/supermercado/category/Des...,Despensa,Pastas y Salsas,ver todo
3,https://www.lider.cl/supermercado/category/Des...,Despensa,Pastas y Salsas,ver todo
4,https://www.lider.cl/supermercado/category/Des...,Despensa,Pastas y Salsas,ver todo
...,...,...,...,...
56,https://www.lider.cl/supermercado/category/Des...,Despensa,Aceites y Aderezos,ver todo
57,https://www.lider.cl/supermercado/category/Des...,Despensa,Aceites y Aderezos,ver todo
58,https://www.lider.cl/supermercado/category/Des...,Despensa,Aceites y Aderezos,ver todo
59,https://www.lider.cl/supermercado/category/Des...,Despensa,Aceites y Aderezos,ver todo


In [22]:
def get_info_product(pais:str, list_pages_category:list):

    driver = get_driver()

    url_category = list()
    name_product = list()
    url_product = list()
    first_level_category = list()
    second_level_category = list()
    third_level_category = list()

    driver.get(list_pages_category[0])

    try:

        wait(driver, 32).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[@id='root']/div[@class='wrapper']/div[1]/div[2]/div[@class='app-container app-container--category-page']/div[@class='bs__grid__container']/div[@class='shop-wrapper']/div[@class='app-container--category-page__walstore-back-button-container']/div[@class='mb-4 ']/div[@class='shop-wrapper']/div[@class='d-flex']/div[@class='col-lg-9 col-md-8 bs__product-list__container']/div[@class='shop-content']/div[@class='ais-Hits']/ul[@class='ais-Hits-list']/li[@class='ais-Hits-item']/div[1]/div[1]")))
        sku = driver.find_elements("xpath", "/html/body/div[@id='root']/div[@class='wrapper']/div[1]/div[2]/div[@class='app-container app-container--category-page']/div[@class='bs__grid__container']/div[@class='shop-wrapper']/div[@class='app-container--category-page__walstore-back-button-container']/div[@class='mb-4 ']/div[@class='shop-wrapper']/div[@class='d-flex']/div[@class='col-lg-9 col-md-8 bs__product-list__container']/div[@class='shop-content']/div[@class='ais-Hits']/ul[@class='ais-Hits-list']/li[@class='ais-Hits-item']/div[1]/div[1]")

        for j in sku:
            url_category.append(list_pages_category[0])

            wait(j, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class=' product-info']/h2/div[@class='product-card_description-wrapper']/div[1]/span[2]")))
            name_product.append(j.find_elements("xpath", "div[@class=' product-info']/h2/div[@class='product-card_description-wrapper']/div[1]/span[2]")[0].text)

            wait(j, 4).until(ec.presence_of_element_located((By.XPATH, "a")))
            url_product.append(j.find_elements("xpath", "a")[0].get_attribute("href"))
            
            first_level_category.append(list_pages_category[1])
            second_level_category.append(list_pages_category[2])
            third_level_category.append(list_pages_category[3])

    except:
        print('ERROR: url not found')
        
    driver.close()
    
    df_url_sku = pd.DataFrame()

    df_url_sku['url_category'] = url_category
    df_url_sku['name_product'] = name_product
    df_url_sku['url_product'] = url_product
    df_url_sku['first_level_category'] = first_level_category
    df_url_sku['second_level_category'] = second_level_category
    df_url_sku['third_level_category'] = third_level_category

    return(df_url_sku.drop_duplicates())

In [23]:
get_info_product('CL', list_pages_categories[1]).head(3)

,url_category,name_product,url_product,first_level_category,second_level_category,third_level_category
0,https://www.lider.cl/supermercado/category/Des...,Pack 3 Spaghetti + 1 Salsa de Tomates Pomarola.,https://www.lider.cl/supermercado/product/sku/...,Despensa,Pastas y Salsas,ver todo
1,https://www.lider.cl/supermercado/category/Des...,"Fideos Caracoquesos Caja., 296 g",https://www.lider.cl/supermercado/product/sku/...,Despensa,Pastas y Salsas,ver todo
2,https://www.lider.cl/supermercado/category/Des...,"Quífaros., 400 g",https://www.lider.cl/supermercado/product/sku/...,Despensa,Pastas y Salsas,ver todo


In [24]:
def get_info_products(pais:str, df_info_category:list):

    results = Parallel(n_jobs=-1, prefer="threads")(delayed(get_info_product)(pais, i) for i in tqdm.tqdm(df_info_category))
    
    return(pd.concat(results).drop_duplicates())

In [25]:
df_url_sku = get_info_products('CL', list_pages_categories)

100%|██████████| 61/61 [02:24<00:00,  2.37s/it]


In [45]:
df_url_sku.head(3)

,url_category,name_product,url_product,first_level_category,second_level_category,third_level_category
0,https://www.lider.cl/supermercado/category/Des...,"Salsa tomate tradicional, 200 g",https://www.lider.cl/supermercado/product/sku/...,Despensa,Pastas y Salsas,ver todo
1,https://www.lider.cl/supermercado/category/Des...,"Fideos Espirales, 400 g",https://www.lider.cl/supermercado/product/sku/...,Despensa,Pastas y Salsas,ver todo
2,https://www.lider.cl/supermercado/category/Des...,"Salsa de tomate italiana, 200 g",https://www.lider.cl/supermercado/product/sku/...,Despensa,Pastas y Salsas,ver todo


In [46]:
df_url_sku.iloc[1,2]

'https://www.lider.cl/supermercado/product/sku/464768/lider-fideos-espirales-400-g'

In [47]:
def validate_url_cat(total_url_sku:pd.DataFrame, df_categories:pd.DataFrame):

    cat_validate = total_url_sku[['url_category']].drop_duplicates()
    cat_validate['url_down'] = 0

    cat_validate = df_categories.merge(cat_validate.rename(columns={'url_category': 'url_third_level_category'}), how='left')
    cat_validate['url_down'] = cat_validate.url_down.fillna(1)

    return(cat_validate)

In [52]:
validate_url_cat(df_url_sku, df_url_cat)

,first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,url_down
0,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Cortas,https://www.lider.cl/supermercado/category/Des...,1.0
1,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Largas,https://www.lider.cl/supermercado/category/Des...,1.0
2,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Lasañas y Rellenos,https://www.lider.cl/supermercado/category/Des...,1.0
3,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Instantáneas,https://www.lider.cl/supermercado/category/Des...,1.0
4,Despensa,Pastas y Salsas,https://www.lider.cl/supermercado/category/Des...,Pastas Integrales,https://www.lider.cl/supermercado/category/Des...,1.0
...,...,...,...,...,...,...
815,Campañas,Precios Que No Se Tocan,https://www.lider.cl/supermercado/category/Cam...,Carnes y Cecinas,https://www.lider.cl/supermercado/category/Cam...,1.0
816,Campañas,Precios Que No Se Tocan,https://www.lider.cl/supermercado/category/Cam...,Perfumería y Bebé,https://www.lider.cl/supermercado/category/Cam...,1.0
817,Campañas,Precios Que No Se Tocan,https://www.lider.cl/supermercado/category/Cam...,Frescos y Lácteos,https://www.lider.cl/supermercado/category/Cam...,1.0
818,Campañas,Precios Que No Se Tocan,https://www.lider.cl/supermercado/category/Cam...,Limpieza del Hogar,https://www.lider.cl/supermercado/category/Cam...,1.0


In [53]:
def get_scraping_sku(pais_sku:str, url_sku:list):

    driver = get_driver()

    driver.get(url_sku[0])

    try:

        wait(driver, 32).until(ec.presence_of_element_located((By.XPATH, "/html/body/div[@id='root']/div[@class='wrapper']/div[1]/div[@class='app-container']/div[@class='rct-app-content']/div[@class='rct-page']/div[@class='content-wrapper']/div[@class='shop-wrapper']/div[@class='rct-page']/div[@class='rct-page-content']/div[@class='shop-wrapper']/div[1]")))
        df_producto = driver.find_elements("xpath", "/html/body/div[@id='root']/div[@class='wrapper']/div[1]/div[@class='app-container']/div[@class='rct-app-content']/div[@class='rct-page']/div[@class='content-wrapper']/div[@class='shop-wrapper']/div[@class='rct-page']/div[@class='rct-page-content']/div[@class='shop-wrapper']/div[1]")[0]

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-detail']")))
            id_url = df_producto.find_elements("xpath", "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-detail']")[0].text
        except:
            id_url = ''

        try: 
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/h1")))
            name_url = df_producto.find_elements("xpath", "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/h1")[0].text
        except:
            name_url = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto product-detail__card-section rct-block product-detail__ok-to-shop']/div[1]/p[2]")))
            description_url =  df_producto.find_elements("xpath", "div[@class='m-auto product-detail__card-section rct-block product-detail__ok-to-shop']/div[1]/p[2]")[0].text
        except:
            description_url = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__brand']/a")))
            atr_sku =  df_producto.find_elements("xpath", "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__brand']/a")[0].text
        except:
            atr_sku = ''
            
        try:

            normal_price = '' 
            internet_price = ''
            cmr_price = ''
            
            try:
                wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-prices']/div[@class='d-flex pdp-mobile-reference-price']/span[@class='saving__price__pdp']")))
                normal_price = df_producto.find_elements("xpath", "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-prices']/div[@class='d-flex pdp-mobile-reference-price']/span[@class='saving__price__pdp']")[0].text
            
            except:
                normal_price = ''

            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-prices']/div[@class='d-flex']/span[@class='pdp-mobile-sales-price']")))
            internet_price = df_producto.find_elements("xpath", "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-prices']/div[@class='d-flex']/span[@class='pdp-mobile-sales-price']")[0].text

        except:
            normal_price = ''
            internet_price = ''
            cmr_price = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-detail']")))
            id_client_sku = df_producto.find_elements("xpath", "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-detail']")[0].text
        except:
            id_client_sku = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-detail']")))
            cod_sku = df_producto.find_elements("xpath", "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-card__product']/div[@class='product-detail-card__product-info']/div[@class='product-detail-card__product-detail']")[0].text
        except:
            cod_sku = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-page__gallery-container']/div[@class='desktop__container']/div[1]/div[1]/div[1]/div[1]/div/img")))
            images = df_producto.find_elements("xpath", "div[@class='m-auto pb-20 product-detail__card-section']/div[@class='product-detail-page__container']/div[1]/div[1]/div[@class='product-detail-card__product-detail-container']/div[@class='product-detail-page__gallery-container']/div[@class='desktop__container']/div[1]/div[1]/div[1]/div[1]/div/img")
            
            img_url = ''
            px_url = ''

            n_imagenes = len(images)

            for im, i in zip(images, range(n_imagenes)):

                try:

                    url_img = Request(im.get_attribute("src"), headers={'User-Agent': 'Mozilla/5.0'})

                    u = urlopen(url_img)
                    raw_data = u.read()
                    u.close()

                    img = Image.open(BytesIO(raw_data))

                    if i == n_imagenes-1:
                        img_url = img_url + im.get_attribute("src")
                        px_url = px_url + str(img.size) 
                
                    else:
                        img_url = img_url + im.get_attribute("src") + '; '
                        px_url = px_url + str(img.size) + '; '

                except:
                    
                    if i == n_imagenes-1:

                        img_url = img_url + 'url_image not found'
                        px_url = px_url + ''

                    else:

                        img_url = img_url + 'url_image not found' + '; '
                        px_url = px_url + '; '
            
            image_info_1, image_info_2 = img_url, px_url

        except:
            image_info_1,image_info_2 = '',''

        try:
            first = url_sku[1]
        except:
            first = ''
        
        try:
            second = url_sku[2]
        except:
            second = ''
        
        try:
            third = url_sku[3]
        except:
            third = ''
        
        try:
            category_1 = url_sku[1]
        except:
            category_1 = ''

        try:
            category_2 = url_sku[2]
        except:
            category_2 = ''

    except:
        
        id_url = ''
        name_url = ''
        description_url = ''
        atr_sku = ''
        normal_price = ''
        internet_price = ''
        cmr_price = ''
        id_client_sku = ''
        cod_sku = ''
        image_info_1 = ''
        image_info_2 = ''
        first = ''
        second = ''
        third = ''
        category_1 = ''
        category_2 = ''
    
    driver.close()

    return pais_sku, url_sku[0], id_url, name_url, description_url, id_client_sku, cod_sku, first, second, third, category_1, category_2, atr_sku, normal_price, internet_price, cmr_price, image_info_1, image_info_2

In [54]:
lista = df_url_sku[['url_product', 'first_level_category', 'second_level_category', 'third_level_category']].values.tolist()
lista

[['https://www.lider.cl/supermercado/product/sku/217/pomarola-salsa-tomate-tradicional-200-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/464768/lider-fideos-espirales-400-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/3369/malloa-salsa-de-tomate-italiana-200-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/2685/talliani-fideos-spaghetti-5-con-huevos-frescos-bolsa-400-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/3363/lider-pack-de-6-salsas-de-tomates-italiana-1200-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/648806/lider-fideos-tallarines-n-77-bolsa-400-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/2689/talliani-fideos-spirali-con-huevos-frescos-bolsa-4

In [55]:
lista[3]

['https://www.lider.cl/supermercado/product/sku/2685/talliani-fideos-spaghetti-5-con-huevos-frescos-bolsa-400-g',
 'Despensa',
 'Pastas y Salsas',
 'ver todo']

In [56]:
get_scraping_sku('CL', lista[3])

('CL',
 'https://www.lider.cl/supermercado/product/sku/2685/talliani-fideos-spaghetti-5-con-huevos-frescos-bolsa-400-g',
 'item 405218',
 'Fideos Spaghetti 5 con Huevos Frescos Bolsa, 400 g',
 'Pasta Al Huevo Seca Spaghetti 5 Talliani, 400g',
 'item 405218',
 'item 405218',
 'Despensa',
 'Pastas y Salsas',
 'ver todo',
 'Despensa',
 'Pastas y Salsas',
 'Talliani',
 '',
 '$1.050',
 '',
 'https://images.lider.cl/wmtcl?source=url[file:/productos/2685a.jpg]&sink',
 '(800, 800)')

In [57]:
def get_df_scraping(pais:str, list_url:list):

    results = Parallel(n_jobs=-1, prefer="threads")(delayed(get_scraping_sku)(pais, li) for li in tqdm.tqdm(list_url))

    return(pd.DataFrame(results, columns=['pais', 'url_sku', 'id', 'name', 'description', 'id_client', 'cod_sku', 'first_category',
                                        'second_category', 'third_category', 'category_1', 'category_2', 'attributes', 'normal_price',
                                        'internet_price', 'cmr_price', 'url_image', 'pixel_image']))

In [58]:
lista[:5]

[['https://www.lider.cl/supermercado/product/sku/217/pomarola-salsa-tomate-tradicional-200-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/464768/lider-fideos-espirales-400-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/3369/malloa-salsa-de-tomate-italiana-200-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/2685/talliani-fideos-spaghetti-5-con-huevos-frescos-bolsa-400-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo'],
 ['https://www.lider.cl/supermercado/product/sku/3363/lider-pack-de-6-salsas-de-tomates-italiana-1200-g',
  'Despensa',
  'Pastas y Salsas',
  'ver todo']]

In [59]:
df_final = get_df_scraping('CL', lista[:5])

100%|██████████| 5/5 [00:00<00:00, 4078.48it/s]


In [60]:
df_final.head(3)

,pais,url_sku,id,name,description,id_client,cod_sku,first_category,second_category,third_category,category_1,category_2,attributes,normal_price,internet_price,cmr_price,url_image,pixel_image
0,CL,https://www.lider.cl/supermercado/product/sku/...,item 311058,"Salsa tomate tradicional, 200 g",Pack 6 x Salsa de Tomates Pomarola Italiana Ca...,item 311058,item 311058,Despensa,Pastas y Salsas,ver todo,Despensa,Pastas y Salsas,Pomarola,,$3.790,,https://images.lider.cl/wmtcl?source=url[file:...,"(800, 800)"
1,CL,https://www.lider.cl/supermercado/product/sku/...,item 377240,"Fideos Espirales, 400 g",Pasta de Sémola de Trigo Vitaminizada Seca Esp...,item 377240,item 377240,Despensa,Pastas y Salsas,ver todo,Despensa,Pastas y Salsas,Lider,$770,$620,,https://images.lider.cl/wmtcl?source=url[file:...,"(800, 800); (800, 800); (800, 800)"
2,CL,https://www.lider.cl/supermercado/product/sku/...,item 264083,"Salsa de tomate italiana, 200 g","Pack 6 x Salsa de Tomate Italiana Malloa, 1200g",item 264083,item 264083,Despensa,Pastas y Salsas,ver todo,Despensa,Pastas y Salsas,Malloa,,$3.890,,https://images.lider.cl/wmtcl?source=url[file:...,"(1000, 1000)"
